# Finetune Mistral-7B-Instruct with QLoRA for Text-to-SQL

This notebook walks through **end-to-end fine-tuning** of
`mistralai/Mistral-7B-Instruct-v0.1` for **Text-to-SQL** using:

- The preprocessed instruction-tuning dataset in `data/processed/*.jsonl`.
- **QLoRA** (4-bit quantization + LoRA adapters).
- **Unsloth** + **bitsandbytes** for efficient training.
- **TRL**'s `SFTTrainer` for supervised fine-tuning.

We assume you are running this in a GPU environment (e.g., Google Colab).

## What you will learn

1. How QLoRA works conceptually and why it is well-suited for 7B models.
2. How schema-grounded prompts (including `CREATE TABLE` context) help
   reduce schema hallucinations in Text-to-SQL.
3. How to load and inspect the preprocessed dataset.
4. How to format prompts for supervised fine-tuning.
5. How to configure and run a QLoRA training loop using Unsloth + TRL.
6. How to save adapters and run a quick sanity check on generated SQL.
7. What the next steps are for external validation and deployment.


## A. Overview: QLoRA + Schema-Grounded Prompts

**QLoRA** (Quantized LoRA) combines:

- **4-bit quantization** of the base model weights (via bitsandbytes).
- **Low-Rank Adapters (LoRA)** that are trained while keeping the
  quantized base model frozen.

This dramatically reduces memory usage, allowing us to fine-tune 7B models
on a single GPU while still achieving strong performance.

For **Text-to-SQL**, we want the model to be:

- **Schema-aware**: It should use only the tables and columns that actually
  exist.
- **Grounded**: The model should see the schema (e.g., `CREATE TABLE`)
  alongside the question.
- **Cautious**: Avoid hallucinating non-existent tables/columns.

To support this, our preprocessing pipeline constructs prompts like:

```text
### Instruction:
Write a SQL query that answers the user's question using ONLY the tables and columns provided in the schema.

### Input:
### Schema:
CREATE TABLE head (age INTEGER)

### Question:
How many heads of the departments are older than 56 ?

### Response:
SELECT COUNT(*) FROM head WHERE age > 56
```

The **instruction** is fixed, the **input** combines schema + question, and
the **response** is the target SQL. We will train the model to map
instruction+input to response.

## B. Environment Setup

This section installs dependencies (for Colab), checks GPU availability, and
sets random seeds for reproducibility.

> If you are running locally with the repository already installed, you may
> skip the `pip install` step and simply import the libraries.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os

# Define the target path
target_path = '/content/drive/MyDrive/Projects/analytics_copilot'

# Change the directory
os.chdir(target_path)

# Verify the change
print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: /content/drive/MyDrive/Projects/analytics_copilot


In [11]:
pip install -q -r /content/drive/MyDrive/Projects/analytics_copilot/requirements.txt

In [12]:
print(type(Path.cwd()))
print(Path.cwd())

<class 'pathlib.PosixPath'>
/content/drive/MyDrive/Projects/analytics_copilot


In [13]:
# If running in Colab, uncomment the following lines to install dependencies.
# !pip install -q unsloth bitsandbytes accelerate transformers datasets trl peft

import os
import json
import random
from pathlib import Path

import torch
from datasets import Dataset
from trl import SFTConfig, SFTTrainer
from unsloth import FastLanguageModel

BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data" / "processed"
OUTPUT_DIR = BASE_DIR / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def set_seed(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

print(f"Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
if not torch.cuda.is_available():
    print("WARNING: CUDA not available. Training a 7B model will not be feasible on CPU.")

Using device: cuda


## C. Load Processed Dataset (train/val JSONL)

The preprocessing step (`scripts/build_dataset.py`) produces two files:

- `data/processed/train.jsonl`
- `data/processed/val.jsonl`

Each line is an Alpaca-style record with keys: `id`, `instruction`, `input`,
`output`, `source`, and `meta`.

We will load them into `datasets.Dataset` objects for use with TRL's
`SFTTrainer`.

In [14]:
def load_alpaca_jsonl(path: Path) -> Dataset:
    if not path.is_file():
        raise FileNotFoundError(f"JSONL file not found: {path}")
    records = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            records.append(json.loads(line))
    return Dataset.from_list(records)

train_path = DATA_DIR / "train.jsonl"
val_path = DATA_DIR / "val.jsonl"

train_raw = load_alpaca_jsonl(train_path)
val_raw = load_alpaca_jsonl(val_path)

print(train_raw)
print(val_raw)

train_raw[0]

Dataset({
    features: ['id', 'instruction', 'input', 'output', 'source', 'meta'],
    num_rows: 920
})
Dataset({
    features: ['id', 'instruction', 'input', 'output', 'source', 'meta'],
    num_rows: 80
})


{'id': 'sqlcc-train-000000',
 'instruction': "Write a SQL query that answers the user's question using ONLY the tables and columns provided in the schema.",
 'input': '### Schema:\nCREATE TABLE genres (id VARCHAR, name VARCHAR); CREATE TABLE tracks (name VARCHAR, genre_id VARCHAR, media_type_id VARCHAR); CREATE TABLE media_types (id VARCHAR, name VARCHAR)\n\n### Question:\nList the name of tracks belongs to genre Rock and whose media type is MPEG audio file.',
 'output': 'SELECT T2.name FROM genres AS T1 JOIN tracks AS T2 ON T1.id = T2.genre_id JOIN media_types AS T3 ON T3.id = T2.media_type_id WHERE T1.name = "Rock" AND T3.name = "MPEG audio file"',
 'source': 'b-mc2/sql-create-context',
 'meta': {'from_local_input': False,
  'original_split': 'train',
  'row': 0,
  'seed': 42,
  'split': 'train',
  'val_ratio': 0.08}}

## D. Prompt Formatting

Our training format combines `instruction`, `input`, and `output` into a
single `text` field. The prompt structure is:

```text
### Instruction:
<instruction>

### Input:
<schema + question>

### Response:
<SQL output>
```

We will implement two helpers mirroring the library code in
`src/text2sql/training/formatting.py`:

- `build_prompt(instruction, input)` – builds the instruction + input + response header.
- `ensure_sql_only(output)` – normalizes whitespace and strips code fences.


In [15]:
import re

PROMPT_TEMPLATE = """### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"""

def build_prompt(instruction: str, input_text: str) -> str:
    instruction = (instruction or "").strip()
    input_text = (input_text or "").strip()
    return PROMPT_TEMPLATE.format(instruction=instruction, input=input_text)

def ensure_sql_only(output: str) -> str:
    if output is None:
        return ""
    text = output.strip()
    if text.startswith("```"):
        text = re.sub(r"^```(?:sql)?\s*", "", text, flags=re.IGNORECASE)
        text = re.sub(r"\s*```$", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

def format_example(example):
    prompt = build_prompt(example["instruction"], example["input"])
    sql = ensure_sql_only(example["output"])
    return {"text": prompt + sql}

formatted_sample = format_example(train_raw[0])
print(formatted_sample["text"][:1000])

### Instruction:
Write a SQL query that answers the user's question using ONLY the tables and columns provided in the schema.

### Input:
### Schema:
CREATE TABLE genres (id VARCHAR, name VARCHAR); CREATE TABLE tracks (name VARCHAR, genre_id VARCHAR, media_type_id VARCHAR); CREATE TABLE media_types (id VARCHAR, name VARCHAR)

### Question:
List the name of tracks belongs to genre Rock and whose media type is MPEG audio file.

### Response:
SELECT T2.name FROM genres AS T1 JOIN tracks AS T2 ON T1.id = T2.genre_id JOIN media_types AS T3 ON T3.id = T2.media_type_id WHERE T1.name = "Rock" AND T3.name = "MPEG audio file"


We can now apply this formatting to the entire dataset. During development,
it is often useful to use a **fast dev run** on a subset of the data.

In [16]:
FAST_DEV_RUN = True  # Set to False for full training
MAX_DEV_SAMPLES = 512

if FAST_DEV_RUN:
    train_raw_sliced = train_raw.select(range(min(MAX_DEV_SAMPLES, train_raw.num_rows)))
    val_raw_sliced = val_raw.select(range(min(MAX_DEV_SAMPLES, val_raw.num_rows)))
else:
    train_raw_sliced = train_raw
    val_raw_sliced = val_raw

train_ds = train_raw_sliced.map(format_example, remove_columns=train_raw_sliced.column_names)
val_ds = val_raw_sliced.map(format_example, remove_columns=val_raw_sliced.column_names)

print(train_ds)
print(val_ds)

Map:   0%|          | 0/512 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 512
})
Dataset({
    features: ['text'],
    num_rows: 80
})


## E. Load Base Model in 4-Bit & Apply LoRA Adapters

We now load `mistralai/Mistral-7B-Instruct-v0.1` using Unsloth's
`FastLanguageModel` in 4-bit mode and attach LoRA adapters.

Key hyperparameters:

- `r` (LoRA rank): capacity of the adapter (typical range: 8–64).
- `alpha`: scale factor for the adapter.
- `dropout`: dropout applied to the adapter; often 0.0 or small.
- `target_modules`: which projection layers receive LoRA (q_proj, k_proj, etc.).


In [17]:
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.1"
MAX_SEQ_LENGTH = 2048
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0.0

if not torch.cuda.is_available():
    print("CUDA is not available. The following cell will likely OOM on CPU.")

dtype = None  # Let Unsloth choose bf16/fp16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

model

==((====))==  Unsloth 2026.1.2: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2026.1.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

## F. Training Loop with TRL's SFTTrainer

We will now configure and run supervised fine-tuning with TRL's `SFTTrainer`.

For **fast dev runs**, we will:

- Limit the number of steps (e.g., `max_steps=20`).
- Use a small subset of the data.

Once everything works, you can disable `FAST_DEV_RUN` and increase
`max_steps` for a proper training run.

In [20]:
MAX_STEPS = 20 if FAST_DEV_RUN else 500
PER_DEVICE_TRAIN_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 8
LEARNING_RATE = 2e-4
WARMUP_STEPS = 50
WEIGHT_DECAY = 0.0

sft_config = SFTConfig(
    output_dir=str(OUTPUT_DIR),
    max_steps=MAX_STEPS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=max(MAX_STEPS // 10, 1),
    save_strategy="steps",
    save_steps=max(MAX_STEPS // 10, 1),
    bf16=torch.cuda.is_bf16_supported(),
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    dataset_text_field="text",
    args=sft_config,
)

trainer

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/512 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/80 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [21]:
train_output = trainer.train()
train_output

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 512 | Num Epochs = 1 | Total steps = 20
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Step,Training Loss,Validation Loss
2,No log,1.851302
4,No log,1.795874
6,No log,1.685717
8,No log,1.540466
10,1.640100,1.364707
12,1.640100,1.161243
14,1.640100,0.929596
16,1.640100,0.784554
18,1.640100,0.734083
20,0.966200,0.690343


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=20, training_loss=1.3031651020050048, metrics={'train_runtime': 288.3895, 'train_samples_per_second': 0.555, 'train_steps_per_second': 0.069, 'total_flos': 866423428276224.0, 'train_loss': 1.3031651020050048, 'epoch': 0.3125})

## G. Save Artifacts (Adapters + Metrics)

After training, we save the LoRA adapters and simple metrics:

- `outputs/adapters/` – adapter weights and tokenizer config.
- `outputs/run_meta.json` – run configuration and dataset sizes.
- `outputs/metrics.json` – train/eval losses and related metrics (if available).

In [22]:
adapters_dir = OUTPUT_DIR / "adapters"
adapters_dir.mkdir(parents=True, exist_ok=True)

trainer.model.save_pretrained(str(adapters_dir))
tokenizer.save_pretrained(str(adapters_dir))

# Save run metadata
run_meta = {
    "base_model": BASE_MODEL,
    "max_steps": MAX_STEPS,
    "per_device_train_batch_size": PER_DEVICE_TRAIN_BATCH_SIZE,
    "gradient_accumulation_steps": GRADIENT_ACCUMULATION_STEPS,
    "learning_rate": LEARNING_RATE,
    "max_seq_length": MAX_SEQ_LENGTH,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "lora_dropout": LORA_DROPOUT,
    "fast_dev_run": FAST_DEV_RUN,
    "num_train_examples": len(train_ds),
    "num_val_examples": len(val_ds),
}

with (OUTPUT_DIR / "run_meta.json").open("w", encoding="utf-8") as f:
    json.dump(run_meta, f, indent=2)

# Save metrics if available
metrics = {}
if hasattr(train_output, "metrics") and train_output.metrics is not None:
    metrics.update(train_output.metrics)

try:
    eval_metrics = trainer.evaluate()
    metrics.update({f"eval_{k}": v for k, v in eval_metrics.items()})
except Exception as e:  # noqa: BLE001
    print("Evaluation failed or was skipped:", e)

with (OUTPUT_DIR / "metrics.json").open("w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2)

metrics

{'train_runtime': 288.3895,
 'train_samples_per_second': 0.555,
 'train_steps_per_second': 0.069,
 'total_flos': 866423428276224.0,
 'train_loss': 1.3031651020050048,
 'epoch': 0.3125,
 'eval_eval_loss': 0.6903432607650757,
 'eval_eval_runtime': 9.6483,
 'eval_eval_samples_per_second': 8.292,
 'eval_eval_steps_per_second': 2.073,
 'eval_epoch': 0.3125}

## H. Quick Sanity Inference

Finally, we perform a quick sanity check by generating SQL for a few
examples. This is a qualitative check to see if the model learned to
produce reasonable SQL given the schema and question.

In [23]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

def generate_sql(example, max_new_tokens: int = 128):
    prompt = build_prompt(example["instruction"], example["input"])
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    streamer = TextStreamer(tokenizer)

    print("=== Prompt ===")
    print(prompt)
    print("=== Generated SQL ===")
    _ = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
    )
    print("\n=================\n")

for i in range(3):
    generate_sql(train_raw[i])

=== Prompt ===
### Instruction:
Write a SQL query that answers the user's question using ONLY the tables and columns provided in the schema.

### Input:
### Schema:
CREATE TABLE genres (id VARCHAR, name VARCHAR); CREATE TABLE tracks (name VARCHAR, genre_id VARCHAR, media_type_id VARCHAR); CREATE TABLE media_types (id VARCHAR, name VARCHAR)

### Question:
List the name of tracks belongs to genre Rock and whose media type is MPEG audio file.

### Response:

=== Generated SQL ===
<s> ### Instruction:
Write a SQL query that answers the user's question using ONLY the tables and columns provided in the schema.

### Input:
### Schema:
CREATE TABLE genres (id VARCHAR, name VARCHAR); CREATE TABLE tracks (name VARCHAR, genre_id VARCHAR, media_type_id VARCHAR); CREATE TABLE media_types (id VARCHAR, name VARCHAR)

### Question:
List the name of tracks belongs to genre Rock and whose media type is MPEG audio file.

### Response:
SELECT tracks.name FROM tracks JOIN genres ON tracks.genre_id = genres

## I. Next Steps

You now have a QLoRA-finetuned Mistral-7B-Instruct model for Text-to-SQL.
The next steps for this project are:

1. **External Validation on Spider dev (planned Task 4)**
   - Evaluate the model on a harder, multi-table, cross-domain benchmark
     such as `xlangai/spider`.
   - Measure logical form accuracy and execution accuracy.
   - See `docs/external_validation.md` for the high-level plan.

2. **Push to Hugging Face Hub (planned Task 5)**
   - Package the LoRA adapters and associated config.
   - Publish a model card documenting training data, metrics, and usage.

3. **Streamlit Remote Inference UI (planned Task 6)**
   - Integrate the fine-tuned model into a Streamlit app.
   - Allow users to connect to a database, ask natural-language questions,
     and inspect / edit the generated SQL.

These steps will complete the full loop from data → training → evaluation →
interactive Analytics Copilot experience.